# Exploratory data analysis

In [2]:
import numpy as np
import pandas as pd
from scipy.io import arff

import seaborn as sb

In [3]:
data, meta = arff.loadarff('autos.arff')

In [25]:
converters = dict()
_ = [converters.update({column:dtype}) for column,dtype in zip(meta.names(),meta.types())]

In [35]:
df = pd.DataFrame(data=data, columns=meta.names())

In [14]:
df.head()

,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,...,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price,symboling
0,NaN,b'alfa-romero',b'gas',b'std',b'two',b'convertible',b'rwd',b'front',88.6,168.8,...,b'mpfi',3.47,2.68,9.0,111.0,5000.0,21.0,27.0,13495.0,b'3'
1,NaN,b'alfa-romero',b'gas',b'std',b'two',b'convertible',b'rwd',b'front',88.6,168.8,...,b'mpfi',3.47,2.68,9.0,111.0,5000.0,21.0,27.0,16500.0,b'3'
2,NaN,b'alfa-romero',b'gas',b'std',b'two',b'hatchback',b'rwd',b'front',94.5,171.2,...,b'mpfi',2.68,3.47,9.0,154.0,5000.0,19.0,26.0,16500.0,b'1'
3,164.0,b'audi',b'gas',b'std',b'four',b'sedan',b'fwd',b'front',99.8,176.6,...,b'mpfi',3.19,3.40,10.0,102.0,5500.0,24.0,30.0,13950.0,b'2'
4,164.0,b'audi',b'gas',b'std',b'four',b'sedan',b'4wd',b'front',99.4,176.6,...,b'mpfi',3.19,3.40,8.0,115.0,5500.0,18.0,22.0,17450.0,b'2'


In [40]:
df.dtypes.unique()

array([dtype('float64'), dtype('O')], dtype=object)